PostgreSQLのデータベースを作成しておく

In [7]:
! pip install langchain-postgres psycopg langchain-ollama langchain langgraph

In [8]:
from langchain_postgres import PGVector
from langchain_postgres.vectorstores import PGVector
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="bge-m3"
)

#パスワードはかけていない
connection = "postgresql+psycopg://tn@localhost:5432/testdb3" 
collection_name = "my_docs"

vectorstore = PGVector(
    embeddings=embedding,
    collection_name=collection_name,
    connection=connection,
    use_jsonb=True,
)

In [3]:
# langchainが作成したテーブルの確認
!psql -d testdb -c "\dt"

                List of relations
 Schema |          Name           | Type  | Owner 
--------+-------------------------+-------+-------
 public | langchain_pg_collection | table | tn
 public | langchain_pg_embedding  | table | tn
(2 rows)



In [4]:
!psql -d testdb -c "\d langchain_pg_embedding"

               Table "public.langchain_pg_embedding"
    Column     |       Type        | Collation | Nullable | Default 
---------------+-------------------+-----------+----------+---------
 id            | character varying |           | not null | 
 collection_id | uuid              |           |          | 
 embedding     | vector            |           |          | 
 document      | character varying |           |          | 
 cmetadata     | jsonb             |           |          | 
Indexes:
    "langchain_pg_embedding_pkey" PRIMARY KEY, btree (id)
    "ix_cmetadata_gin" gin (cmetadata jsonb_path_ops)
    "ix_langchain_pg_embedding_id" UNIQUE, btree (id)
Foreign-key constraints:
    "langchain_pg_embedding_collection_id_fkey" FOREIGN KEY (collection_id) REFERENCES langchain_pg_collection(uuid) ON DELETE CASCADE



以下は　https://github.com/langchain-ai/langchain-postgres/blob/main/examples/vectorstore.ipynb
と https://api.python.langchain.com/en/latest/vectorstores/langchain_postgres.vectorstores.PGVector.html
を参考にした


Initialize the vectorstore

### Drop tables
If you need to drop tables (e.g., updating the embedding to a different dimension or just updating the embedding provider):

vectorstore.drop_tables()

### Add documents
Add documents to the vectorstore

In [9]:
# id:10を修正、id:11を追加する　残すもの全てを書き込まないと消去される
docs = [
    Document(page_content='there are cats in the pond', metadata={"id": 1, "location": "pond", "topic": "animals"}),
    Document(page_content='ducks are also found in the pond', metadata={"id": 2, "location": "pond", "topic": "animals"}),
    Document(page_content='fresh apples are available at the market', metadata={"id": 3, "location": "market", "topic": "food"}),
    Document(page_content='the market also sells fresh oranges', metadata={"id": 4, "location": "market", "topic": "food"}),
    Document(page_content='the new art exhibit is fascinating', metadata={"id": 5, "location": "museum", "topic": "art"}),
    Document(page_content='a sculpture exhibit is also at the museum', metadata={"id": 6, "location": "museum", "topic": "art"}),
    Document(page_content='a new coffee shop opened on Main Street', metadata={"id": 7, "location": "Main Street", "topic": "food"}),
    Document(page_content='the book club meets at the library', metadata={"id": 8, "location": "library", "topic": "reading"}),
    Document(page_content='the library hosts a weekly story time for kids', metadata={"id": 9, "location": "library", "topic": "reading"}),
    Document(page_content='there are tigers in the yard', metadata={"id": 10, "location": "zoo", "topic": "animals"}),
    Document(page_content='there are dogs in the backyard', metadata={"id": 11, "location": "my home", "topic": "animals"})
]

In [10]:
vectorstore.add_documents(docs, ids=[doc.metadata['id'] for doc in docs])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [7]:
vectorstore.similarity_search('lion', k=5)

[Document(id='10', metadata={'id': 10, 'topic': 'animals', 'location': 'zoo'}, page_content='there are tigers in the yard'),
 Document(id='11', metadata={'id': 11, 'topic': 'animals', 'location': 'my home'}, page_content='there are dogs in the backyard'),
 Document(id='2', metadata={'id': 2, 'topic': 'animals', 'location': 'pond'}, page_content='ducks are also found in the pond'),
 Document(id='1', metadata={'id': 1, 'topic': 'animals', 'location': 'pond'}, page_content='there are cats in the pond'),
 Document(id='8', metadata={'id': 8, 'topic': 'reading', 'location': 'library'}, page_content='the book club meets at the library')]

In [11]:
results = vectorstore.similarity_search_with_score(query="lion",k=5)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.457508] there are tigers in the yard [{'id': 10, 'topic': 'animals', 'location': 'zoo'}]
* [SIM=0.494071] there are dogs in the backyard [{'id': 11, 'topic': 'animals', 'location': 'my home'}]
* [SIM=0.540048] ducks are also found in the pond [{'id': 2, 'topic': 'animals', 'location': 'pond'}]
* [SIM=0.541976] there are cats in the pond [{'id': 1, 'topic': 'animals', 'location': 'pond'}]
* [SIM=0.557055] the book club meets at the library [{'id': 8, 'topic': 'reading', 'location': 'library'}]


In [9]:
vectorstore.similarity_search('lion', k=5, filter={
    'topic': { "$eq": 'animals'}
})


[Document(id='10', metadata={'id': 10, 'topic': 'animals', 'location': 'zoo'}, page_content='there are tigers in the yard'),
 Document(id='11', metadata={'id': 11, 'topic': 'animals', 'location': 'my home'}, page_content='there are dogs in the backyard'),
 Document(id='2', metadata={'id': 2, 'topic': 'animals', 'location': 'pond'}, page_content='ducks are also found in the pond'),
 Document(id='1', metadata={'id': 1, 'topic': 'animals', 'location': 'pond'}, page_content='there are cats in the pond')]

### データベースからデータを取得する

ここでカーネルを再起動してメモリ内をクリア

In [5]:
!pip install psycopg

In [12]:
import psycopg

conn = psycopg.connect("dbname=testdb3 user=tn")
cur = conn.cursor()
cur.execute('select * from langchain_pg_embedding')
for row in cur:
    formatted_output = f"id: {row[0]}\n" \
                    f"uuid: {row[1]}\n" \
                    f"page_content: {row[2][:100]}...\n" \
                    f"page_content(string): {row[3]}\n" \
                    f"metadata: {row[4]}\n"
    print(formatted_output)
cur.close()
conn.close()

id: 1
uuid: bce98a32-56ad-4124-94ad-22086ed5fb2e
page_content: [-0.041045193,0.009569716,-0.093480445,0.01990515,0.00062993786,-0.057626557,-0.02735376,-0.01263911...
page_content(string): there are cats in the pond
metadata: {'id': 1, 'topic': 'animals', 'location': 'pond'}

id: 2
uuid: bce98a32-56ad-4124-94ad-22086ed5fb2e
page_content: [-0.0365837,0.0019632874,-0.0848764,-0.007010041,-0.028483586,-0.027577631,-1.8776509e-05,-0.0053111...
page_content(string): ducks are also found in the pond
metadata: {'id': 2, 'topic': 'animals', 'location': 'pond'}

id: 3
uuid: bce98a32-56ad-4124-94ad-22086ed5fb2e
page_content: [0.02102992,0.006635084,-0.05879326,-0.004522216,-0.0065848893,-0.03556204,0.009028944,0.03192697,0....
page_content(string): fresh apples are available at the market
metadata: {'id': 3, 'topic': 'food', 'location': 'market'}

id: 4
uuid: bce98a32-56ad-4124-94ad-22086ed5fb2e
page_content: [-0.021443207,0.001050144,-0.07157226,0.009703566,9.951767e-05,0.0027933442,-0.01428185

In [13]:
!pg_dump -U tn -h localhost -p 5432 -d testdb3 -t langchain_pg_embedding -f langchain_pg_embedding_dump.sql


pg_dump: error: aborting because of server version mismatch
pg_dump: detail: server version: 17.0 (Homebrew); pg_dump version: 16.4


In [19]:
!cat langchain_pg_embedding_dump.sql

--
-- PostgreSQL database dump
--

-- Dumped from database version 16.4 (Homebrew)
-- Dumped by pg_dump version 16.4

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

SET default_tablespace = '';

SET default_table_access_method = heap;

--
-- Name: langchain_pg_embedding; Type: TABLE; Schema: public; Owner: tn
--

CREATE TABLE public.langchain_pg_embedding (
    id character varying NOT NULL,
    collection_id uuid,
    embedding public.vector,
    document character varying,
    cmetadata jsonb
);


ALTER TABLE public.langchain_pg_embedding OWNER TO tn;

--
-- Data for Name: langchain_pg_embedding; Type: TABLE DATA; Schema: public; Owner: tn
--

COPY public.langchain_pg_embedding (id, collection_id, 

In [ ]:
results = vectorstore.similarity_search_with_score(query="lion",k=5)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

----------------------------------------

### 作成したDBを削除

In [4]:
!dropdb -f testdb

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
#from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

# List of URLs to load documents from
urls = [
    "https://thehackernews.com/2024/09/north-korean-hackers-target-energy-and.html",
    "https://thehackernews.com/2022/09/north-korean-hackers-weaponizing-open.html",
]
# Load documents from the URLs
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
# Initialize a text splitter with specified chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
# Split the documents into chunks
doc_splits = text_splitter.split_documents(docs_list)
# Embedding
embedding = OllamaEmbeddings(
    model="bge-m3"
)


### pgvectorで検索

In [30]:
# テーブルを新規に作り直し
vectorstore.drop_tables()